In [1]:
import practice2
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
directory = "Practice_03_data/"
list_data = practice2.list_file_data(directory)
filename = str(directory+list_data[0])

## Exercise 2: Collection Statistics 

In [3]:
pl_1, lt_1 = practice2.text_mining(filename,False)

In [4]:
stat_1 = practice2.get_statistics(pl_1, lt_1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
dl = pd.DataFrame(stat_1['df'],columns=['Document numbers', 'DL'])
tl = pd.DataFrame(stat_1['tl'],columns=['Terms', 'tl'])

In [6]:
c_freq_l = list()
for k,v in stat_1['colec_freq'].items():
    c_freq_l.append((k,v[0][0],v[0][1]))

In [7]:
c_freq = pd.DataFrame(c_freq_l,columns=['Terms', 'freq','Df'])

In [8]:
tl_red = tl[:100]
c_f_red = c_freq[:150]

In [9]:
writer = pd.ExcelWriter('statistique.xlsx')
dl.to_excel(writer,'Document lenght')
tl_red.to_excel(writer,'Terms lenght')
c_f_red.to_excel(writer,'Collection frequency terms')
writer.save()

## Exercise 3: Collection Statistics using stop-words and stemmer 

In [10]:
pl_2, lt_2 = practice2.text_mining(filename,True)


In [14]:
stat_2 = practice2.get_statistics(pl_2,lt_2)
#pl_2 = take(100, pl_2.items())


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
dl = pd.DataFrame(stat_2['df'],columns=['Document numbers', 'DL'])
tl = pd.DataFrame(stat_2['tl'],columns=['Terms', 'tl'])

c_freq_l = list()
i=0
for k,v in stat_2['colec_freq'].items():
    i+=1
    if i==200: break
    c_freq_l.append((k,v[0][0],v[0][1]))

c_freq = pd.DataFrame(c_freq_l,columns=['Terms', 'freq','Df'])

tl_red = tl[:100]
c_f_red = c_freq[:150]

writer = pd.ExcelWriter('statistique_stopword_stemming.xlsx')
dl.to_excel(writer,'Document lenght')
tl_red.to_excel(writer,'Terms lenght')
c_f_red.to_excel(writer,'Collection frequency terms')
writer.save()

## Exercise 4: SMART ltn weighting 

In [16]:
def smart_ltn(posting_list,n):
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value = {} # key doc number, value (terme, ltn)
    for term, value in posting_list.items():
        df =len(value) #
        for v in value:
            tf = v[1]
            if df == 0:
                ltn=0
            else:
                ltn = (1+math.log(tf))*math.log(n/df)
            ltn_value.setdefault(v[0],[]).append((term,ltn))
    return ltn_value

s_ltn = smart_ltn(pl_2, stat_2['n_doc'])

## Exercise 5: Ranked Retrieval (ltn weighting) 

In [17]:
query = "web ranking scoring algorithm"
# Doc = score(sum(ltn(term)))
def rsv_score(query,ltn):
    rsv_doc = {}
    for term in query.split():
        for doc, values in ltn.items():
            for v in values:
                if term in v[0]:
                    rsv_doc.setdefault(doc,[]).append(v[1]) 
                else: continue
    rsv = [(sum(ltn_list),doc) for doc,ltn_list in rsv_doc.items()]
    return rsv

rsv = rsv_score(query,s_ltn)
#rsv = practice2.scoring_doc(practice2.clean(query, True), posting_list, list_terms)

In [83]:
score = sorted(rsv, reverse=True) # To get the score descending
score = [(doc, ltn) for ltn, doc in score]
score[:10]

[('4698944', 449.68585544312054),
 ('444250', 398.936024569417),
 ('33139', 293.95143902325106),
 ('9071052', 264.3803997107955),
 ('18568', 250.0145026585329),
 ('33120', 232.3370616875538),
 ('7672374', 225.9156690940626),
 ('15308316', 223.64523200641196),
 ('448834', 214.41567654730872),
 ('195870', 210.27835231866374)]

## Exercise 6: SMART ltc weighting 
 - Compute a weighted index based on SMART ltc weighting function. 

In [18]:
def smart_ltc(smart_ltn):
    # ltn = doc:(term, ltn)
    #  (ltn_term/sqrt(sum(each pow(ltn_term,2))on a document))
    ltc_value = {}
    for doc, value in smart_ltn.items():
        pow_ltn_term = [pow(v[1],2) for v in value]
        normalize = math.sqrt(sum(pow_ltn_term))
        for v in value:
            ltc_value.setdefault(doc,[]).append((v[0],v[1]/normalize))
    return ltc_value

s_ltc = smart_ltc(s_ltn)

## Exercise 7: Ranked Retrieval (ltc weighting) 
Compute the score of each document for the query  « web ranking scoring algorithm », using the index based 
on SMART ltc weighting function. Print the list of the ten most relevant documents, and their relevance score

In [19]:
rsv_c = rsv_score(query,s_ltc)
score_c = sorted(rsv_c, reverse=True)
score_c = [(doc, ltc) for ltc, doc in score_c]
score_c[:10]

[('9071052', 2.4316182923266667),
 ('7672374', 2.191647713817547),
 ('4698944', 1.6059559755167894),
 ('195870', 1.5909263855644207),
 ('3103995', 1.5674681087184965),
 ('7576424', 1.5473266829983465),
 ('15686791', 1.514233947049167),
 ('6270108', 1.4911388528838958),
 ('6497220', 1.4832313759131743),
 ('33139', 1.4630890122775557)]

## Exercise 8: BM25 weighting 
Compute a weighted index based on BM25 weighting function. 

In [20]:
def tf_part(posting_list,dl):
    #(tf*(k+1))/(k*((1-b)+b*(dl/avdl))+tf)
    #[  1     ]             [   2   ]
    #              [      3         ]
    #            [         4            ]   
    k = 1.2
    b = 0.75
    dl_ = [dl_ for _, dl_ in dl]
    doc_len = 0
    avdl = (sum(dl_))/len(dl_)
    tf_part_val = {}
    for term, value in posting_list.items():
        for v in value:
            tf = v[1]
            doc_len = [doc_l for docno,doc_l in dl if docno==v[0]]
            bloc_1 = tf*(k+1)
            bloc_2 = doc_len[0]/avdl
            bloc_3 = k * ((1-b) + b * bloc_2)
            bloc_4 = bloc_3+tf       
            tf_part_val.setdefault(v[0],[]).append((term,bloc_1/bloc_4)) 
    return tf_part_val

    
    

In [ ]:
tf_part_ = tf_part(pl_2, stat_2['df'])

In [ ]:
 def idf_part(posting_list,n):
    #log((n-df+0.5)/(df+0.5))
    #    [   1    ] [  2   ]
    idf_part_val ={}
    for term, value in posting_list.items():
        df = len(value)
        bloc_1 = n-df+0.5
        bloc_2 = df + 0.5
        
        idf_part_val.setdefault(term,[]).append(math.log(bloc_1/bloc_2))
        print(idf_part_val)
    return idf_part_val




In [ ]:
def bm25(posting_list, stat):
    tf_part_ = tf_part(posting_list,stat['df'])
    print("tf get")
    idf_part_ = idf_part(posting_list,stat['n_doc'])
    print("idf get") 
    bm25_val = {}
    for term, idf_value in idf_part.items():
        for docno,tf_value in tf_part.items():
            bm25_val.setdefault(docno,[]).append((tf_value[0], tf_value[1]*idf_value[0]))
    
    return bm25_val
        

## Exercise 9: Ranked Retrieval (BM25 weighting) 

In [ ]:
bm25_val = bm25(pl_2,stat_2)

In [ ]:
bm25_score= rsv_score(query,bm25_val)
score_bm25 = sorted(bm25_score, reverse=True)
score_bm25 = [(doc, bm25_) for bm25_, doc in score_bm25]
score_bm25[:10]